<center>

# Implementation of New Message Encryption using Elliptic Curve Cryptography Over Finite Fields
</center>

Used article: [link](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9493519)

## Proposed method parameters

<center>

| Set-up Parameters | *n, a, b, p, G* |
|-------------------|-----------------|
| Key Generator     | *P_B, P_A, n_A, n_B, K* |
| Encryption | *E* = [*c_1c_2c_3...c_l*] |
| Decryption | *M* = [m_1m_2m_3...m_l] |
</center>

## 01. Set-up Parameters

A and B side should agree upon an elliptic curve *E_p(a,b)* over finite fields where *p* is a prime number, *a* and *b* are integers less than *p*.

Points are calculated according to the equation:
$$ y^2 = x^3 + ax + b \mod p$$

Which then needs to satisfy the condition:
$$ 4a^3+27b^2\ne 0 \mod p$$

In [2]:
# Example parameters (a,b,p) = (1, 37, 1286081)
import math

a = 1
b = 37
p = 1286081

# Checking if the equation satisfies the condition equality
condition_val = (4*a**3 + 27*b**2) % p

print(condition_val)

36967


In [30]:
# count points on the finite field E_1286081
def sqrt(val: int):
    return val**(1/2)

def count_y(x, a, b):
    aux = x**3 + a*x + b
    return aux

def point(x, y, a, b, p):
    result = (x**3 + a*x + b - y**2) % p
    if result == 0:
        # print("Point P({},{}) belongs to curve".format(x, y))
        return True
    else:
        # print("Result:", result)
        return False
    
def map_points(a, b, p):
    for y in reversed(range(p+1)):
        print("{:2d}".format(y), end=" ")
        for x in range(p+1):
            result = point(x, y, a, b, p)
            if result == True:
                print("+", end=" ")
            else:
                print(".", end=" ")
        print("")
        
    print("   ", end="")
    for x_label in range(p+1):
        if x_label >= 10:
            print("{:2d}".format(x_label), end="")
        else:
            print("{}".format(x_label), end=" ")

# print(point(5, 8, 0, 7, 17))

map_points(0, 7, 17)

17 . . . + . . . . . . . . . . . . . . 
16 . . . . . . . . . . . . + . . . . . 
15 . . . . . . . . . . + . . . . . . . 
14 . . . . . . . . + . . . . . . . . . 
13 . . . . . . . . . . . . . . . + . . 
12 . + . . . . . . . . . . . . . . . . 
11 . . . . . . + . . . . . . . . . . . 
10 . . + . . . . . . . . . . . . . . . 
 9 . . . . . + . . . . . . . . . . . . 
 8 . . . . . + . . . . . . . . . . . . 
 7 . . + . . . . . . . . . . . . . . . 
 6 . . . . . . + . . . . . . . . . . . 
 5 . + . . . . . . . . . . . . . . . . 
 4 . . . . . . . . . . . . . . . + . . 
 3 . . . . . . . . + . . . . . . . . . 
 2 . . . . . . . . . . + . . . . . . . 
 1 . . . . . . . . . . . . + . . . . . 
 0 . . . + . . . . . . . . . . . . . . 
   0 1 2 3 4 5 6 7 8 9 1011121314151617

In [31]:
from tinyec.ec import SubGroup, Curve

field = SubGroup(p=17, g=(12, 16), n=18, h=1)
curve = Curve(a=0, b=7, field=field, name='p1707')
print('curve:', curve)

for k in range(0, 25):
    p = k * curve.g
    print(f"{k} * G = ({p.x}, {p.y})")

curve: "p1707" => y^2 = x^3 + 0x + 7 (mod 17)
0 * G = (None, None)
1 * G = (12, 16)
2 * G = (1, 5)
3 * G = (5, 8)
4 * G = (2, 10)
5 * G = (2, 7)
6 * G = (5, 9)
7 * G = (1, 12)
8 * G = (12, 1)
9 * G = (None, None)
10 * G = (12, 16)
11 * G = (1, 5)
12 * G = (5, 8)
13 * G = (2, 10)
14 * G = (2, 7)
15 * G = (5, 9)
16 * G = (1, 12)
17 * G = (12, 1)
18 * G = (None, None)
19 * G = (12, 16)
20 * G = (1, 5)
21 * G = (5, 8)
22 * G = (2, 10)
23 * G = (2, 7)
24 * G = (5, 9)


### Adding points on a curve

True
